In [8]:
import cv2
import mediapipe as mp
import keyboard
from collections import deque

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

smooth_x = deque(maxlen=5)
smooth_y = deque(maxlen=5)

# Open webcam
cap = cv2.VideoCapture(0)

prev_x, prev_y = None, None
gesture = ""
cooldown = 0  # To avoid spam detection

while True:
    success, frame = cap.read()
    if not success:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    h, w, _ = frame.shape

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

            raw_x = handLms.landmark[0].x
            raw_y = handLms.landmark[0].y

            # Add to buffer
            smooth_x.append(raw_x)
            smooth_y.append(raw_y)

            # Average of last 5 frames
            x = sum(smooth_x) / len(smooth_x)
            y = sum(smooth_y) / len(smooth_y)

            if prev_x is not None:
                dx = x - prev_x
                dy = y - prev_y

                threshold = 0.04

                # if cooldown == 0:  # Only detect if cooldown is 0
                if dx > threshold:
                    gesture = "LEFT"
                    keyboard.press_and_release("left")
                    cooldown = 8

                elif dx < -threshold:
                    gesture = "RIGHT"
                    keyboard.press_and_release("right")
                    cooldown = 8

                elif dy < -threshold:
                    gesture = "UP"
                    keyboard.press_and_release("up")
                    cooldown = 8

                elif dy > threshold:
                    gesture = "DOWN"
                    keyboard.press_and_release("down")
                    cooldown = 8

            prev_x, prev_y = x, y

            # Debug texts
            cv2.putText(frame, f"Gesture: {gesture}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    else:
        prev_x, prev_y = None, None
        gesture = ""

    # # Reduce cooldown gradually
    # if cooldown > 0:
    #     cooldown -= 1

    cv2.imshow("Gesture Control", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()
